## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 8th Dec. 15th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-9-2024,Condo/Co-op,1800 N Bayshore Dr #4108,Miami,FL,33132.0,450000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1800-N-Bayshor...,MARMLS,A11678502,N,Y,25.79391,-80.186611


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
89,PAST SALE,2024-12-13,Condo/Co-op,2600 Island Blvd #701,Aventura,FL,33160.0,1500000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11593206,N,Y,25.939020,-80.133755
25,PAST SALE,2024-12-13,Condo/Co-op,301 174th St #2412,Sunny Isles Beach,FL,33160.0,580000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11683949,N,Y,25.937377,-80.130117
4,PAST SALE,2024-12-13,Condo/Co-op,180 Royal Palm Rd #116,Hialeah,FL,33016.0,175000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/180-Royal-Pa...,Beaches MLS,F10472118,N,Y,25.868209,-80.331240
5,PAST SALE,2024-12-13,Condo/Co-op,7118 Bonita Dr #904,Miami Beach,FL,33141.0,290000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7118-Bon...,Beaches MLS,F10469787,N,Y,25.856422,-80.126682
24,PAST SALE,2024-12-13,Condo/Co-op,9974 SW 88th St #1020,Miami,FL,33176.0,290000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9974-SW-88th-S...,MARMLS,A11688874,N,Y,25.684766,-80.356525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,PAST SALE,2024-12-09,Condo/Co-op,11309 SW 200th St Unit 210-C,Miami,FL,33157.0,197500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11309-SW-200th...,MARMLS,A11644467,N,Y,25.582199,-80.374798
60,PAST SALE,2024-12-09,Condo/Co-op,1040 10th St #304,Miami Beach,FL,33139.0,520000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1040-10t...,MARMLS,A11655193,N,Y,25.780374,-80.139398
46,PAST SALE,2024-12-09,Condo/Co-op,201 Crandon Blvd #641,Key Biscayne,FL,33149.0,1390000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11672277,N,Y,25.696814,-80.158094
55,PAST SALE,2024-12-09,Condo/Co-op,55 SW 9th St #2201,Miami,FL,33130.0,75000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/55-SW-9th-St-3...,MARMLS,A11660819,N,Y,25.765661,-80.194394


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

102


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,002,176


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$564


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$102,222,000


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
94
Input Previous Week Condo Average Sales Price:
846364
Input Previous Week Condo Average PSF:
587
Input Previous Week Condo Sales total (ex: 198_000_000)
79_600_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-12,Condo/Co-op,7034 Fisher Island Dr #7034,Miami Beach,FL,33109.0,14500000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4956.0,NaN,2016.0,NaN,2926.0,10496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,MARMLS,A11642078,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,Dina Goldentayer,Douglas Elliman,None,None,None,None,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-12,Condo/Co-op,7034 Fisher Island Dr #7034,Miami Beach,FL,33109.0,14500000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4956.0,NaN,2016.0,NaN,2926.0,10496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,MARMLS,A11642078,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,Dina Goldentayer,Douglas Elliman,None,None,None,None,None,None,1,orange
1,PAST SALE,2024-12-09,Condo/Co-op,7944 Fisher Island Dr #7944,Miami Beach,FL,33109.0,9500000.0,3.0,3.0,OCEANSIDE @ FISHER ISL CO,3310.0,NaN,1989.0,NaN,2870.0,4546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,MARMLS,A11517820,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,Maria Yudina,Aria Luxe Inc,Evelyn Mackenzie,Aria Luxe Inc,Judith Zeder,Coldwell Banker Realty,None,None,2,blue
2,PAST SALE,2024-12-11,Condo/Co-op,5123 Fisher Island Dr #5123,Miami Beach,FL,33109.0,9165000.0,4.0,4.5,Bayview,3580.0,NaN,2004.0,NaN,2560.0,4448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Beaches MLS,F10436260,N,Y,25.762077,-80.147771,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Miguel Hernandez,One Sotheby's Int'l Realty,Carolina Pueyo,One Sotheby's Int'l Realty,None,None,None,None,3,blue
3,PAST SALE,2024-12-13,Condo/Co-op,2627 S Bayshore Dr #2302,Miami,FL,33133.0,5875000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1459.0,6164.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11634051,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,David Siddons,Douglas Elliman,None,None,Ruedi Sieber,"Sieber International, LLC",None,None,4,blue
4,PAST SALE,2024-12-09,Condo/Co-op,18501 Collins Ave #1903,Sunny Isles Beach,FL,33160.0,5050000.0,3.0,4.5,Turnberry Ocean Club Condo,3110.0,NaN,2020.0,NaN,1624.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11576078,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue
5,PAST SALE,2024-12-10,Condo/Co-op,791 Crandon Blvd #704,Key Biscayne,FL,33149.0,3200000.0,3.0,3.0,OCEAN TOWER TWO CONDO,2510.0,NaN,2001.0,NaN,1275.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11614177,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Douglas Kinsley,Fortune Christie's International Real Estate,None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,6,blue
6,PAST SALE,2024-12-09,Condo/Co-op,2542 Fisher Island Dr #6402,Miami Beach,FL,33109.0,2800000.0,2.0,2.5,BAYSIDE VILLAGE CONDO,1847.0,NaN,1986.0,NaN,1516.0,6841.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,MARMLS,A11531672,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,Karla Abaunza,Luxury Living Realty,Jorge Escasena,Luxury Living Realty,Evelyn Mackenzie,Aria Luxe Inc,None,None,7,blue
7,PAST SALE,2024-12-12,Condo/Co-op,16047 Collins Ave #2202,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,TURNBERRY OCEAN COLONY SO,2245.0,NaN,2007.0,NaN,1158.0,4564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11683657,N,Y,25.925384,-80.121618,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Oksana Ivanenko,"Miami Matrix Realty, LLC.",None,None,Cassio Faccin,FACCIN Group Realty,None,None,8,blue
8,PAST SALE,2024-12-10,Condo/Co-op,18975 Collins Ave #905,Sunny Isles Beach,FL,33160.0,1825000.0,2.0,2.0,18975 COLLINS CONDO,1107.0,NaN,2019.0,NaN,1649.0,1700.0,

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-12,Condo/Co-op,7034 Fisher Island Dr #7034,Miami Beach,FL,33109.0,14500000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4956.0,NaN,2016.0,NaN,2926.0,10496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,MARMLS,A11642078,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,Dina Goldentayer,Douglas Elliman,None,None,None,None,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_12162024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'PALAZZO DEL SOL/DELLA LUN at 7034 Fisher Island Dr #7034 in Miami Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-12,Condo/Co-op,7034 Fisher Island Dr #7034,Miami Beach,FL,33109.0,14500000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4956.0,NaN,2016.0,NaN,2926.0,10496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,MARMLS,A11642078,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,Dina Goldentayer,Douglas Elliman,None,None,None,None,None,None,1,orange,PALAZZO DEL SOL/DELLA LUN at 7034 Fisher Islan...
1,PAST SALE,2024-12-09,Condo/Co-op,7944 Fisher Island Dr #7944,Miami Beach,FL,33109.0,9500000.0,3.0,3.0,OCEANSIDE @ FISHER ISL CO,3310.0,NaN,1989.0,NaN,2870.0,4546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,MARMLS,A11517820,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,Maria Yudina,Aria Luxe Inc,Evelyn Mackenzie,Aria Luxe Inc,Judith Zeder,Coldwell Banker Realty,None,None,2,blue,OCEANSIDE @ FISHER ISL CO at 7944 Fisher Islan...
2,PAST SALE,2024-12-11,Condo/Co-op,5123 Fisher Island Dr #5123,Miami Beach,FL,33109.0,9165000.0,4.0,4.5,Bayview,3580.0,NaN,2004.0,NaN,2560.0,4448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Beaches MLS,F10436260,N,Y,25.762077,-80.147771,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Miguel Hernandez,One Sotheby's Int'l Realty,Carolina Pueyo,One Sotheby's Int'l Realty,None,None,None,None,3,blue,Bayview at 5123 Fisher Island Dr #5123 in Miam...
3,PAST SALE,2024-12-13,Condo/Co-op,2627 S Bayshore Dr #2302,Miami,FL,33133.0,5875000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1459.0,6164.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11634051,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,David Siddons,Douglas Elliman,None,None,Ruedi Sieber,"Sieber International, LLC",None,None,4,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #23...
4,PAST SALE,2024-12-09,Condo/Co-op,18501 Collins Ave #1903,Sunny Isles Beach,FL,33160.0,5050000.0,3.0,4.5,Turnberry Ocean Club Condo,3110.0,NaN,2020.0,NaN,1624.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11576078,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,Turnberry Ocean Club Condo at 18501 Collins Av...
5,PAST SALE,2024-12-10,Condo/Co-op,791 Crandon Blvd #704,Key Biscayne,FL,33149.0,3200000.0,3.0,3.0,OCEAN TOWER TWO CONDO,2510.0,NaN,2001.0,NaN,1275.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11614177,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Douglas Kinsley,Fortune Christie's International Real Estate,None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,6,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #704...
6,PAST SALE,2024-12-09,Condo/Co-op,2542 Fisher Island Dr #6402,Miami Beach,FL,33109.0,2800000.0,2.0,2.5,BAYSIDE VILLAGE CONDO,1847.0,NaN,1986.0,NaN,1516.0,6841.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,MARMLS,A11531672,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,Karla Abaunza,Luxury Living Realty,Jorge Escasena,Luxury Living Realty,Evelyn Mackenzie,Aria Luxe Inc,None,None,7,blue,BAYSIDE VILLAGE CONDO at 2542 Fisher Island Dr...
7,PAST SALE,2024-12-12,Condo/Co-op,16047 Collins Ave #2202,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,TURNBERRY OCEAN COLONY SO,2245.0,NaN,2007.0,NaN,1158.0,4564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16..

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: PALAZZO DEL SOL/DELLA LUN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,800,000M to $14,500,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Miami, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 102 condo sales totaling $102,222,000 million from Dec. 8th to Dec. 15th. The previous week, brokers closed 94 condo sales totaling $79,600,000.

Last week’s units sold for an average of $1,002,176, higher than the $846,364 average price from the previous we

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-12,Condo/Co-op,7034 Fisher Island Dr #7034,Miami Beach,FL,33109.0,14500000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4956.0,NaN,2016.0,NaN,2926.0,10496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,MARMLS,A11642078,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,Dina Goldentayer,Douglas Elliman,None,None,None,None,None,None,1,orange,PALAZZO DEL SOL/DELLA LUN at 7034 Fisher Islan...
1,PAST SALE,2024-12-09,Condo/Co-op,7944 Fisher Island Dr #7944,Miami Beach,FL,33109.0,9500000.0,3.0,3.0,OCEANSIDE @ FISHER ISL CO,3310.0,NaN,1989.0,NaN,2870.0,4546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,MARMLS,A11517820,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,Maria Yudina,Aria Luxe Inc,Evelyn Mackenzie,Aria Luxe Inc,Judith Zeder,Coldwell Banker Realty,None,None,2,blue,OCEANSIDE @ FISHER ISL CO at 7944 Fisher Islan...
2,PAST SALE,2024-12-11,Condo/Co-op,5123 Fisher Island Dr #5123,Miami Beach,FL,33109.0,9165000.0,4.0,4.5,Bayview,3580.0,NaN,2004.0,NaN,2560.0,4448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Beaches MLS,F10436260,N,Y,25.762077,-80.147771,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Miguel Hernandez,One Sotheby's Int'l Realty,Carolina Pueyo,One Sotheby's Int'l Realty,None,None,None,None,3,blue,Bayview at 5123 Fisher Island Dr #5123 in Miam...
3,PAST SALE,2024-12-13,Condo/Co-op,2627 S Bayshore Dr #2302,Miami,FL,33133.0,5875000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1459.0,6164.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11634051,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,David Siddons,Douglas Elliman,None,None,Ruedi Sieber,"Sieber International, LLC",None,None,4,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #23...
4,PAST SALE,2024-12-09,Condo/Co-op,18501 Collins Ave #1903,Sunny Isles Beach,FL,33160.0,5050000.0,3.0,4.5,Turnberry Ocean Club Condo,3110.0,NaN,2020.0,NaN,1624.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11576078,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,Turnberry Ocean Club Condo at 18501 Collins Av...
5,PAST SALE,2024-12-10,Condo/Co-op,791 Crandon Blvd #704,Key Biscayne,FL,33149.0,3200000.0,3.0,3.0,OCEAN TOWER TWO CONDO,2510.0,NaN,2001.0,NaN,1275.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11614177,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Douglas Kinsley,Fortune Christie's International Real Estate,None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,6,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #704...
6,PAST SALE,2024-12-09,Condo/Co-op,2542 Fisher Island Dr #6402,Miami Beach,FL,33109.0,2800000.0,2.0,2.5,BAYSIDE VILLAGE CONDO,1847.0,NaN,1986.0,NaN,1516.0,6841.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,MARMLS,A11531672,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,Karla Abaunza,Luxury Living Realty,Jorge Escasena,Luxury Living Realty,Evelyn Mackenzie,Aria Luxe Inc,None,None,7,blue,BAYSIDE VILLAGE CONDO at 2542 Fisher Island Dr...
7,PAST SALE,2024-12-12,Condo/Co-op,16047 Collins Ave #2202,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,TURNBERRY OCEAN COLONY SO,2245.0,NaN,2007.0,NaN,1158.0,4564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16..

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: PALAZZO DEL SOL/DELLA LUN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,800,000M to $14,500,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Miami, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 102 condo sales totaling $102,222,000 million from Dec. 8th to Dec. 15th. The previous week, brokers closed 94 condo sales totaling $79,600,000.

Last week’s units sold for an average of $1,002,176, higher than the $846,364 average price from the previous we

In [48]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/7034-Fisher-Island-Dr-33109/unit-7034/home/110133179


In [49]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/7944-Fisher-Island-Dr-33109/unit-7944/home/43100922


In [50]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/3131-NE-7th-Ave-33137/unit-4206/home/79136978


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/18975-Collins-Ave-33160/unit-905/home/171996165


In [52]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/7034-Fisher-Island-Dr-33109/unit-7034/home/110133179


In [53]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/3131-NE-7th-Ave-33137/unit-4206/home/79136978


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-12,Condo/Co-op,7034 Fisher Island Dr #7034,Miami Beach,FL,33109.0,14500000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4956.0,NaN,2016.0,NaN,2926.0,10496.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,MARMLS,A11642078,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7034-Fis...,Dina Goldentayer,Douglas Elliman,None,None,None,None,None,None,1,orange,PALAZZO DEL SOL/DELLA LUN at 7034 Fisher Islan...
1,PAST SALE,2024-12-09,Condo/Co-op,7944 Fisher Island Dr #7944,Miami Beach,FL,33109.0,9500000.0,3.0,3.0,OCEANSIDE @ FISHER ISL CO,3310.0,NaN,1989.0,NaN,2870.0,4546.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,MARMLS,A11517820,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7944-Fis...,Maria Yudina,Aria Luxe Inc,Evelyn Mackenzie,Aria Luxe Inc,Judith Zeder,Coldwell Banker Realty,None,None,2,blue,OCEANSIDE @ FISHER ISL CO at 7944 Fisher Islan...
2,PAST SALE,2024-12-11,Condo/Co-op,5123 Fisher Island Dr #5123,Miami Beach,FL,33109.0,9165000.0,4.0,4.5,Bayview,3580.0,NaN,2004.0,NaN,2560.0,4448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Beaches MLS,F10436260,N,Y,25.762077,-80.147771,https://www.redfin.com/FL/Miami-Beach/5123-Fis...,Miguel Hernandez,One Sotheby's Int'l Realty,Carolina Pueyo,One Sotheby's Int'l Realty,None,None,None,None,3,blue,Bayview at 5123 Fisher Island Dr #5123 in Miam...
3,PAST SALE,2024-12-13,Condo/Co-op,2627 S Bayshore Dr #2302,Miami,FL,33133.0,5875000.0,5.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2005.0,NaN,1459.0,6164.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11634051,N,Y,25.732211,-80.234936,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,David Siddons,Douglas Elliman,None,None,Ruedi Sieber,"Sieber International, LLC",None,None,4,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #23...
4,PAST SALE,2024-12-09,Condo/Co-op,18501 Collins Ave #1903,Sunny Isles Beach,FL,33160.0,5050000.0,3.0,4.5,Turnberry Ocean Club Condo,3110.0,NaN,2020.0,NaN,1624.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11576078,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,Turnberry Ocean Club Condo at 18501 Collins Av...
5,PAST SALE,2024-12-10,Condo/Co-op,791 Crandon Blvd #704,Key Biscayne,FL,33149.0,3200000.0,3.0,3.0,OCEAN TOWER TWO CONDO,2510.0,NaN,2001.0,NaN,1275.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11614177,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Douglas Kinsley,Fortune Christie's International Real Estate,None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,6,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #704...
6,PAST SALE,2024-12-09,Condo/Co-op,2542 Fisher Island Dr #6402,Miami Beach,FL,33109.0,2800000.0,2.0,2.5,BAYSIDE VILLAGE CONDO,1847.0,NaN,1986.0,NaN,1516.0,6841.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,MARMLS,A11531672,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2542-Fis...,Karla Abaunza,Luxury Living Realty,Jorge Escasena,Luxury Living Realty,Evelyn Mackenzie,Aria Luxe Inc,None,None,7,blue,BAYSIDE VILLAGE CONDO at 2542 Fisher Island Dr...
7,PAST SALE,2024-12-12,Condo/Co-op,16047 Collins Ave #2202,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,TURNBERRY OCEAN COLONY SO,2245.0,NaN,2007.0,NaN,1158.0,4564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16..

## Time on Market Calculator

In [56]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 1, 19) ## List (Earlier) date
date2 = datetime(2024, 12, 9) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

325


## Clean CSV for Datawrapper Chart

In [57]:
chart_df = df_top_ten

In [58]:
chart_df = chart_df.fillna(" ")

In [59]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [60]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [61]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [62]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [63]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [64]:
csv_date_string = today.strftime("%m_%d_%Y")

In [65]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [66]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-12,7034 Fisher Island Dr #7034 in Miami Beach,"$14,500,000",4,4.5,"4,956",2016,"$2,926",MARMLS,A11642078,Dina Goldentayer Douglas Elliman,
1,2024-12-09,7944 Fisher Island Dr #7944 in Miami Beach,"$9,500,000",3,3.0,"3,310",1989,"$2,870",MARMLS,A11517820,Maria Yudina Aria Luxe Inc Evelyn Mackenzie Ar...,Judith Zeder Coldwell Banker Realty
2,2024-12-11,5123 Fisher Island Dr #5123 in Miami Beach,"$9,165,000",4,4.5,"3,580",2004,"$2,560",Beaches MLS,F10436260,Miguel Hernandez One Sotheby's Int'l Realty Ca...,
3,2024-12-13,2627 S Bayshore Dr #2302 in Miami,"$5,875,000",5,5.5,"4,026",2005,"$1,459",MARMLS,A11634051,David Siddons Douglas Elliman,"Ruedi Sieber Sieber International, LLC"
4,2024-12-09,18501 Collins Ave #1903 in Sunny Isles Beach,"$5,050,000",3,4.5,"3,110",2020,"$1,624",MARMLS,A11576078,Adam Ziefer The Corcoran Group,NON-MLS MEMBER MAR NON MLS MEMBER
5,2024-12-10,791 Crandon Blvd #704 in Key Biscayne,"$3,200,000",3,3.0,"2,510",2001,"$1,275",MARMLS,A11614177,Douglas Kinsley Fortune Christie's Internation...,"Victor Florencio Sanchez Ocean Club Realty, In..."
6,2024-12-09,2542 Fisher Island Dr #6402 in Miami Beach,"$2,800,000",2,2.5,"1,847",1986,"$1,516",MARMLS,A11531672,Karla Abaunza Luxury Living Realty Jorge Escas...,Evelyn Mackenzie Aria Luxe Inc
7,2024-12-12,16047 Collins Ave #2202 in Sunny Isles Beach,"$2,600,000",2,2.5,"2,245",2007,"$1,158",MARMLS,A11683657,"Oksana Ivanenko Miami Matrix Realty, LLC.",Cassio Faccin FACCIN Group Realty
8,2024-12-10,18975 Collins Ave #905 in Sunny Isles Beach,"$1,825,000",2,2.0,"1,107",2019,"$1,649",MARMLS,A11660914,Arabella Conti Miami Waterfront Realty,Brandon Schneider The SIR Group
9,2024-12-12,3131 NE 7th Ave #4206 in Miami,"$1,800,000",3,3.5,"1,634",2018,"$1,102",MARMLS,A11548155,Marko Gojanovic One Sotheby's International Re...,"Mathy Garcia Related ISG Realty, LLC."
